In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers for regression
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='linear')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')

# Data augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and preprocess data
train_generator = datagen.flow_from_directory(
    'path_to_training_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='input'
)

# Train the model
model.fit(train_generator, epochs=50, validation_data=validation_generator)

# Save the model
model.save('NET_counting_model.h5')


If Neutrophil Extracellular Traps (NETs) are already marked with a unique fluorescent color, this simplifies the task of detecting and counting them using a Convolutional Neural Network (CNN). Here’s an optimized pathway to leverage this information for training a CNN:

### Step-by-Step Guide

1. **Data Collection and Annotation**:
   - **Collect Fluorescent Microscopy Images**: Gather a large dataset of microscopy images where NETs are marked with a unique fluorescent marker.
   - **Annotation**: Ensure that images are annotated, if necessary, with the number of NETs or their locations. Given the unique color, this can be automated to some extent.

2. **Data Preparation**:
   - **Color Segmentation**: Pre-process images to isolate the fluorescent color marking the NETs. This can be done using color thresholding techniques.
   - **Resize Images**: Ensure all images are of the same size (e.g., 224x224 pixels) for input into the CNN.
   - **Normalize Pixel Values**: Normalize the pixel values to a range suitable for CNN input (e.g., [0, 1]).

### Example Implementation: Color Segmentation and CNN Training

Here is an example of how to preprocess images using color segmentation and then train a CNN using TensorFlow/Keras.

#### Color Segmentation

First, segment the NETs based on their unique fluorescent color.

```python
import cv2
import numpy as np
from matplotlib import pyplot as plt

def segment_nets(image):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define the color range for the fluorescent marker
    lower_color = np.array([30, 150, 50])
    upper_color = np.array([85, 255, 255])
    
    # Create a mask for the color
    mask = cv2.inRange(hsv, lower_color, upper_color)
    
    # Apply the mask to get the segmented NETs
    result = cv2.bitwise_and(image, image, mask=mask)
    
    return result, mask

# Example usage
image = cv2.imread('path_to_fluorescent_image.jpg')
segmented_image, mask = segment_nets(image)

# Display the original and segmented images
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.subplot(1, 2, 2)
plt.title('Segmented NETs')
plt.imshow(cv2.cvtColor(segmented_image, cv2.COLOR_BGR2RGB))
plt.show()
```

#### CNN Training

Next, use the segmented images to train the CNN.

```python
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load the pre-trained VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='linear')(x)  # For counting, use a single output neuron

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')

# Data augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and preprocess data
train_generator = datagen.flow_from_directory(
    'path_to_training_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'  # Use 'sparse' if labels are integers (counts)
)

validation_generator = datagen.flow_from_directory(
    'path_to_validation_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)

# Train the model
model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator
)

# Save the model
model.save('NET_counting_model.h5')
```

### Steps Breakdown

1. **Color Segmentation**:
   - Use color thresholding to isolate the fluorescently marked NETs. This preprocesses the image to highlight the regions of interest.

2. **Prepare Datasets**:
   - Use segmented images to train the model. The segmented images help the CNN to focus on the NETs, reducing the complexity of learning from raw images.

3. **Train the CNN**:
   - Use the preprocessed (segmented) images to train the CNN. The training process remains similar, but the input data now emphasizes the NETs due to the preprocessing step.

4. **Validation and Testing**:
   - Validate and test the model using similarly preprocessed images to ensure the model's robustness and accuracy.

By using the unique fluorescent marker for NETs, you simplify the detection and counting task for the CNN, which can lead to improved accuracy and efficiency in the training process.

Given that NETs have a unique shape in addition to being marked with a fluorescent marker, you can leverage this information to further enhance the model’s ability to detect and count them. Here's a detailed approach that incorporates shape information along with the fluorescent marker:

### Step-by-Step Guide

1. **Data Collection and Annotation**
   - **Collect Fluorescent Microscopy Images**: Obtain a large dataset of microscopy images where NETs are marked with a unique fluorescent color.
   - **Annotation**: Ensure images are annotated with the number of NETs or their locations.

2. **Data Preparation**
   - **Color Segmentation**: Pre-process images to isolate the fluorescent color marking the NETs.
   - **Shape Feature Extraction**: Use morphological operations to emphasize the unique shapes of NETs.

3. **Data Augmentation**: Apply data augmentation techniques to increase the diversity of the training data.

4. **Model Design**: Use a CNN architecture, potentially enhancing it with custom layers that focus on both color and shape features.

### Implementation Steps

#### Step 1: Color Segmentation and Shape Enhancement

First, segment the NETs based on their unique fluorescent color and then enhance their shapes.

```python
import cv2
import numpy as np
from matplotlib import pyplot as plt

def segment_and_enhance_nets(image):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define the color range for the fluorescent marker
    lower_color = np.array([30, 150, 50])  # Adjust these values based on the color
    upper_color = np.array([85, 255, 255])
    
    # Create a mask for the color
    mask = cv2.inRange(hsv, lower_color, upper_color)
    
    # Apply the mask to get the segmented NETs
    segmented_image = cv2.bitwise_and(image, image, mask=mask)
    
    # Convert the segmented image to grayscale
    gray = cv2.cvtColor(segmented_image, cv2.COLOR_BGR2GRAY)
    
    # Enhance shapes using morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    enhanced = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
    
    return segmented_image, enhanced

# Example usage
image = cv2.imread('path_to_fluorescent_image.jpg')
segmented_image, enhanced_shape = segment_and_enhance_nets(image)

# Display the original, segmented, and enhanced images
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title('Original Image')
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.subplot(1, 3, 2)
plt.title('Segmented NETs')
plt.imshow(cv2.cvtColor(segmented_image, cv2.COLOR_BGR2RGB))
plt.subplot(1, 3, 3)
plt.title('Enhanced Shape')
plt.imshow(enhanced_shape, cmap='gray')
plt.show()
```

#### Step 2: CNN Training

Next, use the segmented and shape-enhanced images to train the CNN.

```python
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, concatenate
from tensorflow.keras.optimizers import Adam

# Define a function to create a combined model
def create_combined_model(input_shape):
    # Load the pre-trained VGG16 model for color features
    base_model_color = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    color_output = base_model_color.output
    color_output = GlobalAveragePooling2D()(color_output)
    
    # Separate input for shape-enhanced images
    shape_input = Input(shape=input_shape)
    base_model_shape = VGG16(weights='imagenet', include_top=False, input_tensor=shape_input)
    shape_output = base_model_shape.output
    shape_output = GlobalAveragePooling2D()(shape_output)
    
    # Concatenate color and shape outputs
    combined = concatenate([color_output, shape_output])
    
    # Add custom layers on top of the combined outputs
    x = Dense(1024, activation='relu')(combined)
    predictions = Dense(1, activation='linear')(x)  # For counting, use a single output neuron
    
    # Define the model
    model = Model(inputs=[base_model_color.input, shape_input], outputs=predictions)
    
    # Freeze the layers of the base models
    for layer in base_model_color.layers:
        layer.trainable = False
    for layer in base_model_shape.layers:
        layer.trainable = False
    
    return model

# Create the model
model = create_combined_model((224, 224, 3))

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')

# Data augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and preprocess data
# Assuming you have prepared directories for segmented and enhanced shape images
train_generator_color = datagen.flow_from_directory(
    'path_to_training_data/segmented',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'  # Use 'sparse' if labels are integers (counts)
)

train_generator_shape = datagen.flow_from_directory(
    'path_to_training_data/enhanced_shape',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)

validation_generator_color = datagen.flow_from_directory(
    'path_to_validation_data/segmented',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)

validation_generator_shape = datagen.flow_from_directory(
    'path_to_validation_data/enhanced_shape',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)

# Train the model
model.fit(
    x=[train_generator_color, train_generator_shape],
    epochs=50,
    validation_data=([validation_generator_color, validation_generator_shape])
)

# Save the model
model.save('NET_counting_model_combined.h5')
```

### Steps Breakdown

1. **Color Segmentation and Shape Enhancement**:
   - Use color thresholding to isolate NETs based on their fluorescent marker.
   - Apply morphological operations to emphasize the unique shapes of NETs.

2. **Prepare Datasets**:
   - Create separate datasets for segmented images and shape-enhanced images.
   - Use these datasets to train the model.

3. **Combined CNN Model**:
   - Use VGG16 as the base model for both color features and shape features.
   - Combine the outputs of both models and add custom layers for the regression task.

4. **Training and Evaluation**:
   - Train the combined model on the prepared datasets.
   - Validate and test the model using similarly preprocessed images.

By leveraging both the unique color and shape of NETs, you can significantly improve the accuracy and robustness of the CNN model for detecting and counting NETs in microscopy images.